## AIHub Json Preprocessing

### Development Environment

In [ ]:
%pip install kss==3.7.3

KSS Argument Error: Restart Jupyter Kernel Runtime

In [ ]:
%pip install python-mecab-ko

KSS 3.7.3 matches python-mecab-ko

In [ ]:
%pip install pandas

In [ ]:
%pip install ray

In [1]:
import re
import os
import kss
import ray
import json
import time
from time import sleep
from tqdm import tqdm
from mecab import MeCab
import pandas as pd
from glob import glob
from itertools import chain

In [30]:
pwd

'd:\\AIHUB'

### AIHUB 논문자료 요약

[Source](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=90)

#### Convert JSON File to TXT File

In [ ]:
from data_preprocessing import make_train_valid_json_txt_file_path_list
from data_preprocessing import count_number_of_txt_file_with_batch_list
from data_preprocessing import write_jsontext_to_txt_file_with_batch_list

In [10]:
json_path_list = ['AIHUB_논문자료 요약/Training/'+ '/**/*.json', 
                  'AIHUB_논문자료 요약/Validation/'+ '/**/*.json']
txt_path_list = ["exploration/summary_of_thesis_materials_pro/AIHUB_summary_of_thesis_materials_train_", 
                 "exploration/summary_of_thesis_materials_pro/AIHUB_summary_of_thesis_materials_valid_"]
corpus_name = "summary_of_thesis_materials"

In [11]:
train_json_file_list, valid_json_file_list, train_txt_file_path_list, valid_txt_file_path_list = \
    make_train_valid_json_txt_file_path_list(json_path_list, txt_path_list, corpus_name)

The number of file: 65


In [21]:
batch_size = 1000
corpus_name = 'summary_of_thesis_materials'
the_number_of_train_txt_file, the_number_of_train_txt_file_list = count_number_of_txt_file_with_batch_list(train_json_file_list, batch_size, corpus_name)

Batch Size: 1000
The number of txt file: 726


In [22]:
source_file_by_batch_train_df = pd.read_excel('source_file_by_batch/summary_of_thesis_materials_train.xlsx', engine='openpyxl')  
source_file_by_batch_train_df

,Unnamed: 0,File,Length of Source List,The Number of txt File,Description
0,0,AIHUB_논문자료 요약/Training\training_특허섹션만\9-N-0.js...,38000,39,NaN
1,1,AIHUB_논문자료 요약/Training\training_특허섹션만\9-N-1.js...,38000,39,NaN
2,2,AIHUB_논문자료 요약/Training\training_특허섹션만\10-N-0.j...,38000,39,NaN
3,3,AIHUB_논문자료 요약/Training\training_논문\논문요약_0206_0...,64000,65,NaN
4,4,AIHUB_논문자료 요약/Training\training_논문\논문요약_0206_1...,64000,65,NaN
5,5,AIHUB_논문자료 요약/Training\training_논문\논문요약_0206_2...,12518,13,NaN
6,6,AIHUB_논문자료 요약/Training\training_논문\논문요약_0220_0...,20042,21,NaN
7,7,AIHUB_논문자료 요약/Training\training_특허전체\특허전체_9-Y-...,6000,7,NaN
8,8,AIHUB_논문자료 요약/Training\training_특허전체\특허전체_9-Y-...,6000,7,NaN
9,9,AIHUB_논문자료 요약/Training\training_특허전체\특허전체_9-Y-...,6000,7,NaN


In [ ]:
batch_size = 1000
corpus_name = 'summary_of_thesis_materials'
write_jsontext_to_txt_file_with_batch_list(train_json_file_list, train_txt_file_path_list, batch_size, the_number_of_train_txt_file_list, corpus_name)

In [23]:
batch_size = 1000
corpus_name = 'summary_of_thesis_materials'
the_number_of_valid_txt_file, the_number_of_valid_txt_file_list = count_number_of_txt_file_with_batch_list(valid_json_file_list, batch_size, corpus_name)

Batch Size: 1000
The number of txt file: 117


In [26]:
source_file_by_batch_valid_df = pd.read_excel('source_file_by_batch/summary_of_thesis_materials_valid.xlsx', engine='openpyxl')  
source_file_by_batch_valid_df

,Unnamed: 0,File,Length of Source List,The Number of txt File,Description
0,0,AIHUB_논문자료 요약/Validation\10-N-1.json ~ AIHUB...,37000,38,NaN
1,1,AIHUB_논문자료 요약/Validation\validation_논문\논문요약_02...,23604,24,NaN
2,2,AIHUB_논문자료 요약/Validation\validation_논문\논문요약_02...,12518,13,NaN
3,3,AIHUB_논문자료 요약/Validation\validation_특허전체\특허전체_...,6000,7,NaN
4,4,AIHUB_논문자료 요약/Validation\validation_특허전체\특허전체_...,6000,7,NaN
5,5,AIHUB_논문자료 요약/Validation\validation_특허전체\특허전체_...,6000,7,NaN
6,6,AIHUB_논문자료 요약/Validation\validation_특허전체\특허전체_...,6000,7,NaN
7,7,AIHUB_논문자료 요약/Validation\validation_특허전체\특허전체_...,6000,7,NaN
8,8,AIHUB_논문자료 요약/Validation\validation_특허전체\특허전체_...,6000,7,NaN


In [ ]:
batch_size = 1000
corpus_name = 'summary_of_thesis_materials'
write_jsontext_to_txt_file_with_batch_list(valid_json_file_list, valid_txt_file_path_list, batch_size, the_number_of_valid_txt_file_list, corpus_name)

#### Preprocess TXT File

In [ ]:
from sentence_segmentation import preprocessing_text
from data_preprocessing import make_pro_post_txt_file_path_list
from data_preprocessing import merge_and_deduplicate_corpus_txt
from reading_data import reading_txt

In [ ]:
pro_corpus_path = "AIHUB_corpus/exploration/summary_of_thesis_materials_pro/AIHUB_summary_of_thesis_materials_" + "*.txt"
pro_post_xlsx_path = "pro_post_corpus_path/summary_of_thesis_materials.xlsx"
pro_total_corpus_path_list, post_total_corpus_path_list = make_pro_post_txt_file_path_list(pro_corpus_path, pro_post_xlsx_path)

In [ ]:
len(pro_total_corpus_path_list)

In [ ]:
pro_coprus_file = pro_total_corpus_path_list[0]
line_length = 1
data_type = "source"

reading_txt(pro_coprus_file, line_length, data_type)

In [ ]:
pro_coprus_file = pro_total_corpus_path_list[0]
line_length = 1
data_type = "preprocessing"

reading_txt(pro_coprus_file, line_length, data_type)

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/summary_of_thesis_materials.xlsx' \
    --start_index 0

In [ ]:
ray.shutdown()

In [ ]:
preprocessing_corpus_path = "AIHUB_corpus/exploration/summary_of_thesis_materials_post/AIHUB_summary_of_thesis_materials_" +"*.txt"
merge_corpus_path = 'AIHUB_corpus/duplicate/AIHUB_summary_of_thesis_materials.txt'
deduplicate_corpus_path = 'AIHUB_corpus/AIHUB_summary_of_thesis_materials.txt'

In [ ]:
merge_and_deduplicate_corpus_txt(preprocessing_corpus_path, merge_corpus_path, 
                                  deduplicate_corpus_path)